In [1]:
ROOT_DIR = r" " # Root directory of the project

In [ ]:
import ultralytics
from ultralytics import YOLO

model = YOLO(r'') #Load the model of the best pt file

results = model(r'') #Load the test image file

for result in results:
    boxes = result.boxes.xyxy.cpu().numpy()  
    scores = result.boxes.conf.cpu().numpy() 
    classes = result.boxes.cls.cpu().numpy() 
    
    class_names = result.names

    with open('output_predictions.txt', 'w') as f:
        for i in range(len(boxes)):
            class_id = int(classes[i])
            class_name = class_names[class_id]
            score = scores[i]
            x1, y1, x2, y2 = boxes[i]
            f.write(f'{class_id} {class_name} {score:.4f} {x1:.2f} {y1:.2f} {x2:.2f} {y2:.2f}\n')

print("Predictions saved to output_predictions.txt")

In [ ]:
import requests
import json

# Ollama API endpoint
OLLAMA_ENDPOINT = " " #Ollama API endpoint

# Read YOLO detection results
with open("output_predictions.txt", "r") as file:
    yolo_results = file.readlines()

# Format the YOLO results properly
formatted_results = []
for line in yolo_results:
    parts = line.strip().split()
    if len(parts) == 7:
        formatted_results.append(f"Detected {parts[1]} with {float(parts[2])*100:.1f}% confidence at coordinates ({parts[3]}, {parts[4]}, {parts[5]}, {parts[6]})")

# Read the prompt template
with open("prompt_template.txt", "r") as f:
    prompt_template = f.read()

# Replace the placeholder with formatted results
prompt = prompt_template.replace("%DETECTIONS%", chr(10).join(formatted_results))

# Request parameters
data = {
    "model": "imagedet2",
    "prompt": prompt,
    "stream": False
}

# Make the API request
response = requests.post(OLLAMA_ENDPOINT, json=data)
response_json = response.json()
print("\nImage Analysis:")
print(response_json['response'])

# Save the interpretation to a file
with open("interpretation_results.txt", "w") as f:
    f.write(response_json['response'])